In [1]:
import urllib
import time
import tqdm
import pandas as pd
import itertools
import urllib.request
import numpy as np
import json
import pickle


import re
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image

import html
import os

#### First source: http://orientalbirdimages.org/birdimages.php

In [2]:
df = pd.read_csv('../../data/oriental_bird_database/labels.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)
df['image_url'] = None
df = df.sample(df.shape[0], random_state=12345, replace=False)

In [3]:
# Don't want to get a bunch of class imbalances, so 
MAX_IMAGES_PER_CLASS = 64

#### how many images does that get me?
* probably reasonable?

In [4]:
vcs = df['bird_id'].value_counts()
vcs[vcs<MAX_IMAGES_PER_CLASS].sum() + 65*np.sum(vcs > MAX_IMAGES_PER_CLASS)

92804

In [5]:
save_path = '../data/oriental_bird_database/images/'

In [6]:
def download_and_save_image(row):
    if download_and_save_image.bird_counter_dict[row['bird_id']] > MAX_IMAGES_PER_CLASS: 
        return row
    else:
        download_and_save_image.bird_counter_dict[row['bird_id']]+=1
    
    
    def func_to_call(row):
        image_save_path = save_path + '%d_%d.jpg'%(row['bird_id'], row['image_id'])
        if os.path.exists(image_save_path):
            return row

        response = requests.get(row['url'])
        time.sleep(1)
        soup = BeautifulSoup(response.text, 'html.parser')
        img_tags = soup.find_all('img')
        urls = [img['src'] for img in img_tags]
        try:
            url_i_want = urls[1]
        except IndexError:
            print('from idx error ', row['url'])
            return row
        row['image_url'] = url_i_want
        response = requests.get(url_i_want)
        img = Image.open(BytesIO(response.content))
        img.save(image_save_path)
        time.sleep(1)
        
    try:
        return func_to_call(row)
    except OSError:
        print('from os error ', row['url'])
        return row
#     except requests.ConnectionResetError:
#         time.sleep(60*20)
#         return func_to_call(row)
    return row


bird_ids = df['bird_id'].unique().tolist()
download_and_save_image.bird_counter_dict = {bird_id : 0 for bird_id in bird_ids}

In [7]:
tqdm.tqdm.pandas()

total shown by tqdm is "wrong". It should be: 92804 (for MAX_IMAGES_PER_CLASS==64)  
not quite right since my code has an off by 1 error. clsoe enough

In [8]:
pd.Series([str(f)[:str(f).find('_')] for f in os.listdir('../data/oriental_bird_database/images/')]).value_counts()

816     65
971     65
871     65
1593    65
740     65
177     65
1895    65
121     65
1962    65
1613    65
2793    65
2000    65
360     65
2688    65
2293    65
1599    65
874     65
2420    65
1885    65
2029    65
2040    65
1032    65
1559    65
2296    65
509     65
575     65
1738    65
2581    65
886     65
1716    65
        ..
1065     1
2162     1
910      1
36       1
2390     1
491      1
3007     1
2391     1
2761     1
2316     1
1474     1
194      1
2071     1
2280     1
1165     1
1136     1
918      1
2460     1
708      1
1277     1
1289     1
2788     1
2905     1
2914     1
222      1
2279     1
1670     1
1742     1
2896     1
992      1
Length: 2940, dtype: int64

In [9]:
df = df.progress_apply(download_and_save_image, axis=1)

  7%|████▉                                                                     | 11360/171711 [00:13<21:25, 124.77it/s]

from os error  http://orientalbirdimages.org/search.php?Bird_ID=891&Bird_Image_ID=2308


 18%|█████████████▋                                                            | 31742/171711 [00:27<08:32, 272.96it/s]

from os error  http://orientalbirdimages.org/search.php?Bird_ID=1003&Bird_Image_ID=9843


 23%|█████████████████                                                         | 39449/171711 [00:34<08:25, 261.45it/s]

...

 91%|█████████████████████████████████████████████████████████████████▎      | 155778/171711 [50:57<6:16:43,  1.42s/it]

from os error  http://orientalbirdimages.org/search.php?Bird_ID=2690&Bird_Image_ID=178947


 99%|███████████████████████████████████████████████████████████████████████▍| 170368/171711 [4:01:22<43:25,  1.94s/it]

from os error  http://orientalbirdimages.org/search.php?Bird_ID=2924&Bird_Image_ID=73941


100%|████████████████████████████████████████████████████████████████████████| 171711/171711 [4:17:58<00:00, 11.09it/s]
